# Data Preparation: Download Core Datasets

This notebook downloads and processes the 5 anchor datasets for Santa Fe geospatial analysis:

1. **Census Tracts + ACS Demographics** - From U.S. Census Bureau
2. **City Parcels + Zoning** - From City of Santa Fe GIS
3. **Hydrology** - From USGS NHD or NM State GIS
4. **OSM Roads + POIs** - From OpenStreetMap
5. **City Limits Boundary** - From Census TIGER/Line or City GIS

## Workflow

1. Download raw data → `data/raw/`
2. Process (reproject, clip) → `data/processed/`
3. Verify data quality


In [2]:
import sys
from pathlib import Path

# Add project root to path so we can import src modules
project_root = Path().resolve().parent.parent
sys.path.insert(0, str(project_root))

import geopandas as gpd
import pandas as pd
from src.data.download import (
    download_census_tracts,
    download_osm_data,
    download_hydrology,
    download_city_parcels,
    download_city_limits,
    process_downloaded_data
)
from src.config import DATA_RAW, DATA_PROCESSED, get_data_path

print(f"Project root: {project_root}")
print(f"Raw data directory: {DATA_RAW}")
print(f"Processed data directory: {DATA_PROCESSED}")


Project root: /Users/richard/Documents/projects/santa-fe
Raw data directory: /Users/richard/Documents/projects/santa-fe/data/raw
Processed data directory: /Users/richard/Documents/projects/santa-fe/data/processed


## 1. Download City Limits (Do This First)

City limits are needed to clip other datasets.


In [3]:
# Download NM places (cities) from Census TIGER/Line
city_limits_raw = download_city_limits()

if city_limits_raw:
    print(f"\n✓ Downloaded to: {city_limits_raw}")
    
    # Process: extract Santa Fe city and save
    if city_limits_raw.suffix == '.zip':
        import zipfile
        import tempfile
        
        with tempfile.TemporaryDirectory() as tmpdir:
            with zipfile.ZipFile(city_limits_raw, 'r') as zip_ref:
                zip_ref.extractall(tmpdir)
            
            shp_files = list(Path(tmpdir).rglob("*.shp"))
            if shp_files:
                places = gpd.read_file(shp_files[0])
                
                # Filter for Santa Fe city (PLACEFP = 70490 or NAME contains "Santa Fe")
                santa_fe = places[
                    (places['PLACEFP'] == '70490') | 
                    (places['NAME'].str.contains('Santa Fe', case=False, na=False))
                ].copy()
                
                if len(santa_fe) > 0:
                    # Set CRS if missing
                    if santa_fe.crs is None:
                        santa_fe = santa_fe.set_crs("EPSG:4326")
                    
                    # Save to processed
                    output_path = get_data_path("city_limits", processed=True)
                    santa_fe.to_file(output_path, driver="GPKG")
                    print(f"✓ City limits processed and saved to: {output_path}")
                else:
                    print("⚠ Could not find Santa Fe city in places data")
                    print("Available places:", places['NAME'].head(10).tolist())
else:
    print("\n⚠ Manual download required. See instructions above.")


Note: Will need to filter for Santa Fe city (PLACEFP=70490)



✓ Downloaded to: /Users/richard/Documents/projects/santa-fe/data/raw/nm_places_2022.zip
✓ City limits processed and saved to: /Users/richard/Documents/projects/santa-fe/data/processed/city_limits.gpkg


## 2. Download Census Tracts + ACS Data


In [5]:
# Download census tracts for Santa Fe County (FIPS: 35049)
tracts_shp, acs_csv = download_census_tracts(
    state_fips="35",  # New Mexico
    county_fips="049",  # Santa Fe County
    year=2022
)

# Process tracts: clip to city limits and reproject
if tracts_shp.exists():
    tracts_gdf = gpd.read_file(tracts_shp)
    
    # Clip to city limits if available
    city_limits_path = get_data_path("city_limits", processed=True)
    if city_limits_path.exists():
        city_limits = gpd.read_file(city_limits_path)
        tracts_gdf = gpd.clip(tracts_gdf, city_limits)
        print(f"✓ Clipped {len(tracts_gdf)} tracts to city limits")
    
    # Join ACS demographic data if available
    if acs_csv.exists():
        print("\nJoining ACS demographic data to tracts...")
        acs_df = pd.read_csv(acs_csv)
        
        # Ensure GEOID column exists in tracts (it should from TIGER/Line)
        if 'GEOID' not in tracts_gdf.columns:
            # Create GEOID from GEOIDFP or similar column
            if 'GEOIDFP' in tracts_gdf.columns:
                tracts_gdf['GEOID'] = tracts_gdf['GEOIDFP']
            elif 'GEOID20' in tracts_gdf.columns:
                tracts_gdf['GEOID'] = tracts_gdf['GEOID20']
            else:
                # Try to construct from state + county + tract FIPS
                state_col = [c for c in tracts_gdf.columns if 'STATEFP' in c.upper()][0] if any('STATEFP' in c.upper() for c in tracts_gdf.columns) else None
                county_col = [c for c in tracts_gdf.columns if 'COUNTYFP' in c.upper()][0] if any('COUNTYFP' in c.upper() for c in tracts_gdf.columns) else None
                tract_col = [c for c in tracts_gdf.columns if 'TRACTFP' in c.upper() or 'TRACTCE' in c.upper()][0] if any('TRACTFP' in c.upper() or 'TRACTCE' in c.upper() for c in tracts_gdf.columns) else None
                
                if state_col and county_col and tract_col:
                    tracts_gdf['GEOID'] = (
                        tracts_gdf[state_col].astype(str).str.zfill(2) +
                        tracts_gdf[county_col].astype(str).str.zfill(3) +
                        tracts_gdf[tract_col].astype(str).str.zfill(6)
                    )
                else:
                    print("⚠ Could not create GEOID for joining. Available columns:", list(tracts_gdf.columns)[:10])
        
        # Join ACS data
        if 'GEOID' in tracts_gdf.columns:
            tracts_gdf = tracts_gdf.merge(acs_df, on='GEOID', how='left')
            print(f"✓ Joined ACS data: {tracts_gdf[acs_df.columns].notna().sum().sum()} values added")
        else:
            print("⚠ Cannot join ACS data: GEOID column missing")
    else:
        print("\n⚠ ACS data not downloaded (check API key in .env file)")
    
    # Reproject to local CRS
    if tracts_gdf.crs is None:
        tracts_gdf = tracts_gdf.set_crs("EPSG:4326")
    tracts_gdf = tracts_gdf.to_crs("EPSG:32113")  # NM State Plane
    
    # Save processed
    output_path = get_data_path("census_tracts", processed=True)
    tracts_gdf.to_file(output_path, driver="GPKG")
    print(f"✓ Processed tracts saved to: {output_path}")
    print(f"  Columns: {list(tracts_gdf.columns)[:10]}...")
else:
    print("⚠ Census tracts download failed")



Note: ACS demographic data requires:
1. Census API key (get from https://api.census.gov/data/key_signup.html)
2. Install census package: pip install census
3. Set CENSUS_API_KEY environment variable

Tracts shapefile saved to: /Users/richard/Documents/projects/santa-fe/data/raw/census_tracts_2022/tl_2022_35_tract.shp
ACS data CSV (to be downloaded): /Users/richard/Documents/projects/santa-fe/data/raw/acs_2022_5yr_santa_fe.csv
✓ Clipped 45 tracts to city limits

⚠ ACS data not downloaded (check API key in .env file)
✓ Processed tracts saved to: /Users/richard/Documents/projects/santa-fe/data/processed/census_tracts_acs.gpkg
  Columns: ['STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'NAME', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER']...


## 3. Download OSM Roads + POIs


In [6]:
# Download OSM data for Santa Fe area
osm_raw = download_osm_data(use_overpass=True)

if osm_raw and osm_raw.exists():
    print(f"\n✓ OSM data downloaded to: {osm_raw}")
    
    # Process OSM JSON to GeoDataFrames
    import json
    from shapely.geometry import Point, LineString
    
    with open(osm_raw, 'r') as f:
        osm_data = json.load(f)
    
    print("\nProcessing OSM data...")
    
    # Extract nodes (for POIs) and ways (for roads)
    nodes = {}
    ways = []
    pois = []
    roads = []
    
    # First pass: collect all nodes
    for element in osm_data.get('elements', []):
        if element['type'] == 'node':
            nodes[element['id']] = {
                'lon': element['lon'],
                'lat': element['lat'],
                'tags': element.get('tags', {})
            }
    
    # Second pass: process ways (roads)
    for element in osm_data.get('elements', []):
        if element['type'] == 'way':
            way_nodes = element.get('geometry', [])
            if way_nodes:
                # Build LineString from way geometry
                coords = [(node['lon'], node['lat']) for node in way_nodes]
                if len(coords) >= 2:
                    geom = LineString(coords)
                    tags = element.get('tags', {})
                    
                    # Check if it's a road
                    if 'highway' in tags:
                        roads.append({
                            'geometry': geom,
                            'highway': tags.get('highway'),
                            'name': tags.get('name', ''),
                            'osm_id': element['id']
                        })
    
    # Process POIs (nodes with tags)
    for node_id, node_data in nodes.items():
        tags = node_data['tags']
        if tags:  # Only nodes with tags are POIs
            pois.append({
                'geometry': Point(node_data['lon'], node_data['lat']),
                'amenity': tags.get('amenity', ''),
                'shop': tags.get('shop', ''),
                'name': tags.get('name', ''),
                'osm_id': node_id
            })
    
    # Create GeoDataFrames
    if roads:
        roads_gdf = gpd.GeoDataFrame(roads, crs="EPSG:4326")
        print(f"  ✓ Extracted {len(roads_gdf)} road segments")
    else:
        roads_gdf = gpd.GeoDataFrame(columns=['geometry', 'highway', 'name', 'osm_id'], crs="EPSG:4326")
    
    if pois:
        pois_gdf = gpd.GeoDataFrame(pois, crs="EPSG:4326")
        print(f"  ✓ Extracted {len(pois_gdf)} POIs")
    else:
        pois_gdf = gpd.GeoDataFrame(columns=['geometry', 'amenity', 'shop', 'name', 'osm_id'], crs="EPSG:4326")
    
    # Combine roads and POIs into single GeoDataFrame
    # Add a 'feature_type' column to distinguish
    roads_gdf['feature_type'] = 'road'
    pois_gdf['feature_type'] = 'poi'
    
    # Combine (keeping all columns)
    combined_osm = pd.concat([
        roads_gdf[['geometry', 'feature_type', 'highway', 'name', 'osm_id']].rename(columns={'highway': 'category'}),
        pois_gdf[['geometry', 'feature_type', 'amenity', 'shop', 'name', 'osm_id']].assign(
            category=pois_gdf['amenity'].fillna(pois_gdf['shop'])
        )[['geometry', 'feature_type', 'category', 'name', 'osm_id']]
    ], ignore_index=True)
    
    # Clip to city limits if available
    city_limits_path = get_data_path("city_limits", processed=True)
    if city_limits_path.exists():
        city_limits = gpd.read_file(city_limits_path)
        combined_osm = gpd.clip(combined_osm, city_limits)
        print(f"  ✓ Clipped to city limits: {len(combined_osm)} features")
    
    # Reproject to local CRS
    if combined_osm.crs is None:
        combined_osm = combined_osm.set_crs("EPSG:4326")
    combined_osm = combined_osm.to_crs("EPSG:32113")
    
    # Save to processed directory
    output_path = get_data_path("osm", processed=True)
    combined_osm.to_file(output_path, driver="GPKG")
    print(f"✓ Processed OSM data saved to: {output_path}")
    print(f"  - Roads: {len(combined_osm[combined_osm['feature_type'] == 'road'])}")
    print(f"  - POIs: {len(combined_osm[combined_osm['feature_type'] == 'poi'])}")
else:
    print("⚠ OSM download failed or returned None")


OSM data saved to: /Users/richard/Documents/projects/santa-fe/data/raw/osm_santa_fe.json

✓ OSM data downloaded to: /Users/richard/Documents/projects/santa-fe/data/raw/osm_santa_fe.json

Processing OSM data...
  ✓ Extracted 10798 road segments
  ✓ Extracted 2234 POIs


/var/folders/q0/mqqgl1bx65v46lkyhq0xb7pm0000gn/T/ipykernel_68730/1494901553.py:93: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4269

  combined_osm = gpd.clip(combined_osm, city_limits)


  ✓ Clipped to city limits: 11142 features
✓ Processed OSM data saved to: /Users/richard/Documents/projects/santa-fe/data/processed/osm_roads_pois.gpkg
  - Roads: 9085
  - POIs: 2057


## 4. Download Hydrology Data


In [9]:
# Download hydrology data (rivers, streams, waterbodies)
# Note: USGS NHD was retired in Oct 2023. Using OSM as default source.

try:
    # Try OSM first (fastest, good coverage for Santa Fe)
    hydro_raw = download_hydrology(source="osm")
    
    if hydro_raw and hydro_raw.exists():
        print(f"\n✓ Hydrology data downloaded to: {hydro_raw}")
        
        # Process OSM hydrology JSON to GeoDataFrame
        print("\nProcessing OSM hydrology data...")
        import json
        from shapely.geometry import Point, LineString, Polygon
        
        with open(hydro_raw, 'r') as f:
            osm_data = json.load(f)
        
        hydro_features = []
        
        # Process OSM elements (ways and relations for water features)
        for element in osm_data.get('elements', []):
            if element['type'] == 'way':
                way_geom = element.get('geometry', [])
                if way_geom:
                    coords = [(node['lon'], node['lat']) for node in way_geom]
                    if len(coords) >= 2:
                        geom = LineString(coords)
                        tags = element.get('tags', {})
                        
                        # Waterways (rivers, streams, canals)
                        if 'waterway' in tags:
                            hydro_features.append({
                                'geometry': geom,
                                'waterway_type': tags.get('waterway'),
                                'name': tags.get('name', ''),
                                'osm_id': element['id'],
                                'feature_type': 'waterway'
                            })
                        # Natural water features
                        elif tags.get('natural') == 'water':
                            # Close the polygon if it's a waterbody
                            if len(coords) > 2 and coords[0] == coords[-1]:
                                geom = Polygon(coords)
                            hydro_features.append({
                                'geometry': geom,
                                'waterway_type': 'waterbody',
                                'name': tags.get('name', ''),
                                'osm_id': element['id'],
                                'feature_type': 'waterbody'
                            })
        
        if hydro_features:
            hydro_gdf = gpd.GeoDataFrame(hydro_features, crs="EPSG:4326")
            print(f"  ✓ Extracted {len(hydro_gdf)} water features")
            
            # Clip to city limits
            city_limits_path = get_data_path("city_limits", processed=True)
            if city_limits_path.exists():
                city_limits = gpd.read_file(city_limits_path)
                hydro_gdf = gpd.clip(hydro_gdf, city_limits)
                print(f"  ✓ Clipped to city limits: {len(hydro_gdf)} features")
            
            # Reproject
            hydro_gdf = hydro_gdf.to_crs("EPSG:32113")
            
            # Save
            output_path = get_data_path("hydrology", processed=True)
            hydro_gdf.to_file(output_path, driver="GPKG")
            print(f"✓ Processed hydrology saved to: {output_path}")
        else:
            print("⚠ No water features found in OSM data")
            print("Try alternative sources:")
            print("  - download_hydrology(source='nm') for NM State GIS")
            print("  - download_hydrology(source='usgs_3dhp') for 3DHP instructions")
    
    else:
        print("\n⚠ OSM hydrology download failed or returned None")
        print("\nThis might be normal - Santa Fe is in a semi-arid region with limited surface water.")
        print("\nOptions:")
        print("1. Create empty hydrology file (for pipeline continuity)")
        print("2. Try alternative sources:")
        print("   - download_hydrology(source='nm') for NM State GIS")
        print("   - download_hydrology(source='usgs_3dhp') for 3DHP instructions")
        print("3. Manually download from:")
        print("   - NM State GIS: https://www.nmgis.org/")
        print("   - USGS 3DHP: https://www.usgs.gov/3d-hydrography-program/access-3dhp-data-products")
        
        # Create empty hydrology file so pipeline can continue
        print("\nCreating empty hydrology file for pipeline continuity...")
        empty_hydro = gpd.GeoDataFrame(
            columns=['geometry', 'waterway_type', 'name', 'osm_id', 'feature_type'],
            crs="EPSG:32113"
        )
        output_path = get_data_path("hydrology", processed=True)
        empty_hydro.to_file(output_path, driver="GPKG")
        print(f"✓ Empty hydrology file created at: {output_path}")
        print("  (You can add data later or skip hydrology analysis)")
    
except Exception as e:
    print(f"⚠ Error downloading hydrology: {e}")
    print("\nCreating empty hydrology file for pipeline continuity...")
    try:
        empty_hydro = gpd.GeoDataFrame(
            columns=['geometry', 'waterway_type', 'name', 'osm_id', 'feature_type'],
            crs="EPSG:32113"
        )
        output_path = get_data_path("hydrology", processed=True)
        empty_hydro.to_file(output_path, driver="GPKG")
        print(f"✓ Empty hydrology file created at: {output_path}")
    except Exception as e2:
        print(f"⚠ Could not create empty file: {e2}")
    
    print("\nManual download options:")
    print("1. OSM: https://www.openstreetmap.org/ (use Overpass API)")
    print("2. NM State GIS: https://www.nmgis.org/")
    print("3. USGS 3DHP: https://www.usgs.gov/3d-hydrography-program/access-3dhp-data-products")



⚠ OSM hydrology download failed or returned None

This might be normal - Santa Fe is in a semi-arid region with limited surface water.

Options:
1. Create empty hydrology file (for pipeline continuity)
2. Try alternative sources:
   - download_hydrology(source='nm') for NM State GIS
   - download_hydrology(source='usgs_3dhp') for 3DHP instructions
3. Manually download from:
   - NM State GIS: https://www.nmgis.org/
   - USGS 3DHP: https://www.usgs.gov/3d-hydrography-program/access-3dhp-data-products

Creating empty hydrology file for pipeline continuity...
✓ Empty hydrology file created at: /Users/richard/Documents/projects/santa-fe/data/processed/hydrology.gpkg
  (You can add data later or skip hydrology analysis)


## 5. Download City Parcels + Zoning

**Note:** This typically requires manual download from the city GIS portal.


In [10]:
# Try to download (will show instructions if manual download needed)
parcels_raw = download_city_parcels()

if parcels_raw and parcels_raw.exists():
    print(f"\n✓ Parcels downloaded to: {parcels_raw}")
    
    # Process parcels
    try:
        process_downloaded_data(
            "parcels",
            parcels_raw,
            clip_to_city=True
        )
        print("✓ Parcels processed successfully")
    except Exception as e:
        print(f"⚠ Error processing parcels: {e}")
else:
    print("\n📋 Manual download instructions shown above.")
    print("After downloading, place file in:", DATA_RAW)
    print("Then run: process_downloaded_data('parcels', path_to_file)")


City of Santa Fe Parcels & Zoning:
1. Visit: https://www.santafenm.gov/gis
2. Look for 'Parcels' or 'Zoning' data layer
3. Download as Shapefile or GeoJSON
4. Save to: /Users/richard/Documents/projects/santa-fe/data/raw/city_parcels_zoning.zip

Alternatively, check:
- ArcGIS Online: https://www.arcgis.com/apps/mapviewer/index.html
- Search for 'Santa Fe parcels'

📋 Manual download instructions shown above.
After downloading, place file in: /Users/richard/Documents/projects/santa-fe/data/raw
Then run: process_downloaded_data('parcels', path_to_file)


## Verify All Datasets

Check that all processed datasets exist and are valid GeoPackage files.

Expected files in `data/processed/`:
- ✅ `city_limits.gpkg`
- ✅ `census_tracts_acs.gpkg` (tracts only; ACS demographics need API key)
- ✅ `hydrology.gpkg`
- ✅ `osm_roads_pois.gpkg`
- ✅ `parcels_zoning.gpkg`


In [ ]:
from src.data.loaders import (
    load_city_limits,
    load_census_tracts,
    load_hydrology,
    load_parcels
)

datasets = {
    "City Limits": load_city_limits,
    "Census Tracts": load_census_tracts,
    "Hydrology": load_hydrology,
    "Parcels": load_parcels,
}

print("Dataset Status:")
print("=" * 50)

for name, loader_func in datasets.items():
    try:
        gdf = loader_func()
        
        # Handle None return (for load_city_limits)
        if gdf is None:
            print(f"✗ {name}: Not found")
        else:
            print(f"✓ {name}: {len(gdf)} features, CRS: {gdf.crs}")
    except FileNotFoundError as e:
        print(f"✗ {name}: Not found - {e}")
    except Exception as e:
        print(f"⚠ {name}: Error - {e}")

print("\n" + "=" * 50)
print("Next: Run 001_who_lives_where.ipynb to start analysis")
